<a href="https://colab.research.google.com/github/Ang3lino/recomenderSys/blob/master/matrixFactorization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:

import numpy as np
import pandas as pd

import os
import random
import pickle

from sortedcontainers import SortedList
from collections import Counter, defaultdict
from tqdm import tqdm  # modulo cuya finalidad es dar un feedback del progreso de algun procedimiento

In [0]:
# !pip install tqdm --upgrade
# tqdm.pandas()

In [118]:
from google.colab import drive  
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
def load_object(fname: str, user_count: int, item_count: int) -> defaultdict:
    fdir = 'drive/My Drive/petroleo/movielens-20m-dataset'
    fname = f'{fname}_{user_count}_{item_count}.json'
    fpath = os.path.join(fdir, 'shrinked', fname)
    with open(fpath, 'rb') as fp:
        object_ = pickle.load(fp)
    return object_

def defaultdict_set(defdict):
    return {k: set(v) for k, v in defdict.items()}


user_count = 4096
item_count = 512
user2item = load_object('user2item', user_count, item_count)
item2user = load_object('item2user', user_count, item_count)
user_item2rating = load_object('user_item2rating', user_count, item_count)



In [0]:
user2item = {int(k): list(map(int, v)) for k, v in user2item.items()}
item2user = {int(k): list(map(int, v)) for k, v in item2user.items()}
user_item2rating = {(int(i), int(j)): v for (i, j), v in user_item2rating.items()}


# Factorizacion de matrices
Con el fin de reducir espacio de almacenamiento y aumentar la velocidad del algoritmo aplicaremos factorizacion de matrices. Aqui, se busca obtener dos matrices cuyo producto aproxime de mejor manera a $R$. Es decir

$$R \approx \hat R = WU^T$$

Asumamos que $R$ con $m$ usuarios y $n$ articulos, donde $W$ es de dimension $m\times k$ y $U$ es de dimension $n \times k$. Definamos tambien la funcion de perdida

$$ J = \sum_{i, j} (r_{ij} - \hat r_{ij})^2 = \sum_{i,j} (r_{ij} - w_i^T u_j)^2 $$ 

Como de costumbre, se busca minimizar la funcion $J$, derivando parcialmente e igualando a cero tenemos.

$$ w_{i} = (\sum_{j\in\psi_i}u_ju_j^T)^{-1} \sum_{j\in\psi_i}r_{ij}u_j $$ 
y $u_j$ como 
$$u_{j} = (\sum_{i\in \Omega_j}w_iw_i^T)^{-1} \sum_{i\in \Omega_j}r_{ij}w_i$$

Vemos que tanto $w_i$ como $u_j$ dependen mutuamente. Resolveremos este problema aplicando el algoritmo de los minimos cuadrados altenantes, inicializamos tanto $U$ como $W$ con valores aleatorios y aplicamos el algoritmos un numero determinado de epochs.

In [0]:
def loss_function(ratings: dict, u, w):
    ''' r[(i, j)] -> int '''
    return np.mean([(r - w[i].dot(u[j]))**2 for (i, j), r in ratings.items()])

def solve_system(dst, src, R, index_relation, k):
    I = index_relation.keys()
    for i in I:
        matrix = np.zeros((k,k))
        vector = np.zeros(k)
        try:
            for j in index_relation[i]:
                v = src[j]
                matrix += np.outer(v, v)
                vector += np.dot(R[(i,j)], v)
            dst[i] = np.linalg.solve(matrix, vector) 
        except KeyError:
            pass

# funciona tambien, pero no minimiza de la mejor manera posible ?
# def compute_matrices(R, i2j, j2i, k, epochs=10):
#     n = int(max(j2i.keys())) + 1
#     assert(k < n)
#     m = int(max(i2j.keys())) + 1
#     W = np.random.randn(m, k)
#     U = np.random.randn(n, k)
#     for epoch in tqdm(range(epochs)):
#         solve_system(W, U, R, i2j, k)
#         solve_system(U, W, R, j2i, k)
#     return U, W



In [0]:
def compute_matrices(R, i2j, j2i, k, epochs=10):
    n = int(max(j2i.keys())) + 1
    assert(k < n)
    m = int(max(i2j.keys())) + 1
    W = np.random.randn(m, k)
    U = np.random.randn(n, k)
    for epoch in tqdm(range(epochs)):
        for i in i2j.keys():
            matrix = np.zeros((k,k))
            vector = np.zeros(k)
            for j in i2j[i]:
                matrix += np.outer(U[j], U[j])
                vector += R[(i,j)] * U[j]
            W[i] = np.linalg.solve(matrix, vector)
        for j in j2i.keys():
            matrix = np.zeros((k,k))
            vector = np.zeros(k)
            try:
                for i in j2i[j]:
                    matrix += np.outer(W[i], W[i])
                    vector += R[(i,j)] * W[i]
                U[j] = np.linalg.solve(matrix, vector)
            except KeyError:
                pass 
    return U, W

In [145]:
U, W = compute_matrices(user_item2rating, user2item, item2user, 10, 10)




  0%|          | 0/10 [00:00<?, ?it/s]


 10%|█         | 1/10 [00:23<03:30, 23.44s/it]


 20%|██        | 2/10 [00:45<03:04, 23.10s/it]


 30%|███       | 3/10 [01:07<02:39, 22.77s/it]


 40%|████      | 4/10 [01:29<02:15, 22.60s/it]


 50%|█████     | 5/10 [01:51<01:51, 22.39s/it]


 60%|██████    | 6/10 [02:14<01:29, 22.38s/it]


 70%|███████   | 7/10 [02:36<01:06, 22.28s/it]


 80%|████████  | 8/10 [02:58<00:44, 22.18s/it]


 90%|█████████ | 9/10 [03:20<00:22, 22.30s/it]


100%|██████████| 10/10 [03:43<00:00, 22.31s/it]




In [146]:
ans = loss_function(user_item2rating, U, W)
print(ans)

0.49050991611722505


In [161]:
test_matrix = {}
m = 10
n = 10
for i in range(m):
    for j in range(n):
        test_matrix[(i, j)] = i*m + j
i2j = {x: [x for x in range(n)] for x in range(m)}
j2i = {x: [x for x in range(m)] for x in range(n)}
print(test_matrix)
U, W = compute_matrices(test_matrix, i2j, j2i, 2, 10)

print(W.shape)
print(U.shape)
print(U.T.shape)
print(np.dot(W, U.T))

ans = loss_function(test_matrix, U, W)
print(ans)




  0%|          | 0/10 [00:00<?, ?it/s]


100%|██████████| 10/10 [00:00<00:00, 346.53it/s]

{(0, 0): 0, (0, 1): 1, (0, 2): 2, (0, 3): 3, (0, 4): 4, (0, 5): 5, (0, 6): 6, (0, 7): 7, (0, 8): 8, (0, 9): 9, (1, 0): 10, (1, 1): 11, (1, 2): 12, (1, 3): 13, (1, 4): 14, (1, 5): 15, (1, 6): 16, (1, 7): 17, (1, 8): 18, (1, 9): 19, (2, 0): 20, (2, 1): 21, (2, 2): 22, (2, 3): 23, (2, 4): 24, (2, 5): 25, (2, 6): 26, (2, 7): 27, (2, 8): 28, (2, 9): 29, (3, 0): 30, (3, 1): 31, (3, 2): 32, (3, 3): 33, (3, 4): 34, (3, 5): 35, (3, 6): 36, (3, 7): 37, (3, 8): 38, (3, 9): 39, (4, 0): 40, (4, 1): 41, (4, 2): 42, (4, 3): 43, (4, 4): 44, (4, 5): 45, (4, 6): 46, (4, 7): 47, (4, 8): 48, (4, 9): 49, (5, 0): 50, (5, 1): 51, (5, 2): 52, (5, 3): 53, (5, 4): 54, (5, 5): 55, (5, 6): 56, (5, 7): 57, (5, 8): 58, (5, 9): 59, (6, 0): 60, (6, 1): 61, (6, 2): 62, (6, 3): 63, (6, 4): 64, (6, 5): 65, (6, 6): 66, (6, 7): 67, (6, 8): 68, (6, 9): 69, (7, 0): 70, (7, 1): 71, (7, 2): 72, (7, 3): 73, (7, 4): 74, (7, 5): 75, (7, 6): 76, (7, 7): 77, (7, 8): 78, (7, 9): 79, (8, 0): 80, (8, 1): 81, (8, 2): 82, (8, 3): 83, (